In [17]:
from PIL import Image

def encode_lsb(image_path, message, output_path):
    """ Koduje wiadomość w obrazie za pomocą algorytmu LSB. """
    img = Image.open(image_path)
    img = img.convert('RGB')
    width, height = img.size
    pixels = list(img.getdata())
    total_pixels = len(pixels)
    
    message_bytes = message.encode('utf-8')
    message_length = len(message_bytes)
    
    # Sprawdź maksymalną długość wiadomości
    max_length = (total_pixels * 3 - 32) // 8
    if message_length > max_length:
        raise ValueError(f"Wiadomość jest za długa. Maksymalna długość: {max_length} bajtów")
    
    # Przygotuj dane binarne
    length_bits = format(message_length, '032b')
    message_bits = ''.join(format(byte, '08b') for byte in message_bytes)
    binary_data = length_bits + message_bits
    total_bits = len(binary_data)
    
    # Oblicz potrzebną liczbę pikseli i dopełnij zerami
    required_pixels = (total_bits + 2) // 3  # Zaokrąglenie w górę
    binary_data += '0' * (required_pixels * 3 - total_bits)
    
    if required_pixels > total_pixels:
        raise ValueError("Obraz jest za mały, by pomieścić wiadomość")
    
    # Zmodyfikuj piksele
    new_pixels = []
    data_index = 0
    for i in range(required_pixels):
        r, g, b = pixels[i]
        
        if data_index < len(binary_data):
            r = (r & 0xFE) | int(binary_data[data_index])
            data_index += 1
        if data_index < len(binary_data):
            g = (g & 0xFE) | int(binary_data[data_index])
            data_index += 1
        if data_index < len(binary_data):
            b = (b & 0xFE) | int(binary_data[data_index])
            data_index += 1
        
        new_pixels.append((r, g, b))
    
    # Dodaj pozostałe piksele bez zmian
    new_pixels += pixels[len(new_pixels):]
    
    # Zapisz nowy obraz
    new_img = Image.new('RGB', (width, height))
    new_img.putdata(new_pixels)
    new_img.save(output_path, 'PNG')

def decode_lsb(image_path):
    """ Dekoduje wiadomość z obrazu zakodowaną algorytmem LSB. """

    img = Image.open(image_path)
    img = img.convert('RGB')
    pixels = list(img.getdata())
    
    # Zbierz wszystkie bity LSB
    all_bits = []
    for pixel in pixels:
        r, g, b = pixel
        all_bits.extend([r & 1, g & 1, b & 1])
    
    # Odczytaj długość wiadomości (pierwsze 32 bity)
    if len(all_bits) < 32:
        raise ValueError("Obraz nie zawiera poprawnej wiadomości")
    
    length_str = ''.join(str(bit) for bit in all_bits[:32])
    message_length = int(length_str, 2)
    
    # Oblicz potrzebną liczbę bitów
    total_bits_needed = 32 + message_length * 8
    if len(all_bits) < total_bits_needed:
        raise ValueError("Uszkodzona lub niepełna wiadomość")
    
    # Wyodrębnij bity wiadomości
    message_bits = all_bits[32:total_bits_needed]
    
    # Konwertuj bity na bajty
    message_bytes = bytearray()
    for i in range(0, len(message_bits), 8):
        byte_bits = message_bits[i:i+8]
        if len(byte_bits) < 8:
            break
        byte = int(''.join(str(bit) for bit in byte_bits), 2)
        message_bytes.append(byte)
    
    # Dekoduj UTF-8
    try:
        return message_bytes.decode('utf-8')
    except UnicodeDecodeError:
        return "Nie można zdekodować wiadomości (nieprawidłowe UTF-8)"

# Przykład użycia
if __name__ == "__main__":
    # Kodowanie
    wiadomosc = "Litwo, Ojczyzno moja! ty jesteś jak zdrowieIle cię trzeba cenić, ten tylko się dowie,Kto cię stracił. Dziś piękność twą w całej ozdobieWidzę i opisuję, bo tęsknię po tobie.Panno święta, co Jasnej bronisz CzęstochowyI w Ostrej świecisz Bramie! Ty, co gród zamkowy"
    encode_lsb("piwo.jpg", wiadomosc, "output.png")
    
    # Dekodowanie
    message = decode_lsb("output.png")
    print("Odkryta wiadomość:", message)

Odkryta wiadomość: Litwo, Ojczyzno moja! ty jesteś jak zdrowieIle cię trzeba cenić, ten tylko się dowie,Kto cię stracił. Dziś piękność twą w całej ozdobieWidzę i opisuję, bo tęsknię po tobie.Panno święta, co Jasnej bronisz CzęstochowyI w Ostrej świecisz Bramie! Ty, co gród zamkowy
